In [2]:
from urllib.parse import urlencode, urlparse
import requests
import time
from pprint import pprint

VERSION = '5.62'
APP_ID = 5940689  # My application ID
VK_TIMEOUT = 1 # sec
MAX_FRIENDS = 700
VK_PAUSE = 90
VK_REQ_LIMIT = 30
FRIENDS_IDS_FILE = 'vk_all_friends_data.csv'
TOKEN_FILE = 'my_vk_access_token.txt'
AUTHORIZE_URL = 'https://oauth.vk.com/authorize'

auth_data = {
    'client_id': APP_ID,
    'display': 'mobile',
    'response_type': 'token',
    'scope': 'friends', 'status'
    'v': VERSION,
}

In [ ]:
token_url = '?'.join((AUTHORIZE_URL, urlencode(auth_data)))
print(token_url)

my_token_url = input('input token url: ')

o = urlparse(my_token_url)
fragments = dict((i.split('=') for i in o.fragment.split('&')))
my_access_token = fragments['access_token']

print ("new token:", my_access_token)

with open(TOKEN_FILE, 'w') as f:
    print(my_access_token,  file=f)

In [ ]:
def store_pool_friends(friends):
    with open(FRIENDS_IDS_FILE, 'a') as csv_file:
         for f in friends:
            print(f[0], f[1], *f[2], sep=',', end='\n', file=csv_file)       

def get_name_and_friend_ids(user_id, token):
        pars = {'access_token': token, 'v': VERSION, 'user_ids':[user_id], 'fields':['counters' ] } 
        resp = requests.get('https://api.vk.com/method/users.get', pars)
        time.sleep(VK_TIMEOUT)
        x = resp.json()
        try:
            xdat = x['response'][0]
        except:
            pprint(x)
            return None, None
        else:
            try:
                fname = xdat['first_name'] + " " + xdat['last_name']
                n_friends = xdat['counters']['friends']
            except:
                pprint(x)
                return None, None
                
        print(user_id, fname, "has", n_friends, 'friends', end=' ')
        
        pars = {'access_token': token, 'v': VERSION, 'user_id':user_id, 'count': MAX_FRIENDS  }
        resp = requests.get('https://api.vk.com/method/friends.get', pars)
        time.sleep(VK_TIMEOUT)
        y = resp.json()
        try:
            ydat = y['response']
        except:
            pprint(y)
            return None, None
        else:
            fr_list  = ydat['items']
        print('loaded', len(fr_list))
        return fname, fr_list

In [ ]:
with open(TOKEN_FILE) as f:
    my_access_token = f.readline()

# 79400816, Катерина Ходакова

# Get first/root list of friends
while True:
    try:
        first_id = int(input("enter 1st vk id: "))
    except ValueError:
        print("id must be a number!")
    else:
        break

first_name, first_friends = get_name_and_friend_ids(first_id, my_access_token)

if first_name != None :
    n_first_friends = len(first_friends)
    print("found user id:", first_id, "name:", first_name, "has", n_first_friends, "friends" )
    print("starting building list of friends friends for analysis.....")
else:
    print("wrong user_id, or invalid access token")

with open(FRIENDS_IDS_FILE, 'w') as csv_file:
    print(first_id, first_name, *first_friends, sep=',', end='\n', file=csv_file) 

requests_pool = [ first_friends[i:i+VK_REQ_LIMIT] for i in range(0, n_first_friends, VK_REQ_LIMIT)]
total_friends = n_first_friends

for pool in requests_pool:
    pool_friends = list()
    processed_ids = 0 
    for friend_id in pool:
        processed_ids += 1 
        fr_name, fr_list_of_friends = get_name_and_friend_ids(friend_id, my_access_token)
        if fr_name == None :
            print ('------- user id', friend_id, 'has no data, ignored --------')
        else:
            pool_friends.append((friend_id, fr_name, fr_list_of_friends))
            total_friends += len(fr_list_friends)
            time.sleep(VK_TIMEOUT)

    store_pool_friends(pool_friends)
    print("processed", processed_ids, "loaded", total_friends, "paused for", VK_PAUSE/60, "mins")
    if processed_ids < n_first_friends :
        time.sleep(VK_PAUSE) 

print("finished, processed", total_friends, "friend's ids")

